# Crear Web API para utilizar modelo entrenado con TensorFlow+Keras usando Flask y ngrok
Fuentes:

https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html

https://deeplizard.com/learn/video/SI1hVGvbbZ4

https://curiousily.com/posts/deploy-keras-deep-learning-project-to-production-with-flask/

https://pyngrok.readthedocs.io/en/latest/integrations.html#google-colaboratory

# Preparar Web API

In [1]:
#@title Accede al Drive

# Acceder al drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [9]:
#@title Cargar Modelo ya entrenado con sus complementos
import os
import joblib
import tensorflow
from tensorflow.keras.models import load_model

path_modelo = '/content/gdrive/My Drive/IA/demoModelDeployment/modelo'  #@param {type:"string"}

scaler = None

# cargar modelo
model = load_model(path_modelo+"/model.keras")
print("\n* Modelo cargado de ", path_modelo, "\n")
model.summary()

# cargar scaler (si existe)
fn_scaler = path_modelo+"/scaler.joblib"
if os.path.isfile(fn_scaler):
  scaler = joblib.load(fn_scaler)
  print("\n* Scaler cargado de ", fn_scaler, "\n")
else:
  scaler = None
  print("\n* Scaler no encontrado en ", fn_scaler, "\n")

fn_clases = path_modelo+"/CLASES.txt"
CLASES = []
if os.path.isfile(fn_clases):
  with open(fn_clases, 'r') as f:
    # carga datos
    auxData = f.readlines()
  for c in auxData:
    CLASES.append( c.replace("\n", "") )
  print("\n* CLASES definidas cargado de ", fn_clases, ":")
  print("\t\t", CLASES, "\n")
else:
  print("\n* CLASES no encontradas en ", fn_clases, "\n")



* Modelo cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo 



Model: "RNA"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidd_1 (Dense)                       │ (None, 12)                  │              60 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 4)                   │              52 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 338 (1.32 KB)

 Trainable params: 112 (448.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 226 (908.00 B)


* Scaler no encontrado en  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/scaler.joblib 


* CLASES definidas cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo/CLASES.txt :
		 ['na', 'Setosa', 'Versicolor', 'Virginica'] 



In [2]:
#@title Instalar pyngrok (opcional)

usar_ngrok_web_publica = True #@param{type:"boolean"}

if usar_ngrok_web_publica:
  # Instalar pyngrok
  !pip install pyngrok
  print("")
else:
  print("- no se usa ngrok.")


In [3]:
#@title Preparar la conexión con ngrok para hacer sitio público (opcional)

ngrok_auth_token = "" #@param {type:"string"}

if usar_ngrok_web_publica:
  import getpass
  from pyngrok import ngrok, conf

  # determina el authentication token de ngrok (
  if (ngrok_auth_token == ""):
    print("Ingrese el authtoken indicada en https://dashboard.ngrok.com/get-started/your-authtoken luego de registrarse")
    conf.get_default().auth_token = getpass.getpass()
  else:
    conf.get_default().auth_token = ngrok_auth_token
  print("")
  # Open a TCP ngrok tunnel to the SSH server
  connection_string = ngrok.connect("22", "tcp").public_url
  print("")
  ssh_url, port = connection_string.strip("tcp://").split(":")
  print("")
  print(f" * se crea ngrok tunnel, accediendo con `ssh root@{ssh_url} -p{port}`")
else:
  print("- no se usa ngrok.")


Ingrese el authtoken indicada en https://dashboard.ngrok.com/get-started/your-authtoken luego de registrarse
··········



 * se crea ngrok tunnel, accediendo con `ssh root@0.tcp.ngrok.io -p13841`


In [4]:
#@title Define default Web Page (opcional)

DEFAULT_WEB_PAGE = '''
<!DOCTYPE html>
<html>

<head>
    <base target="_top">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>

<body>
    <div class="container">
        <h1>Demo para clasificar flores IRIS <br> usando entrenado con TensorFlow+Keras usando Flash y ngrok</h1>
        <br>
        <div class="form-inline">
            <div class="form-group">
                <br>  URL Web API:  <input type="text" size="100" class="form-control" id="URLWebAPI" value=""/>
                <br>
                <span class="mr-2">-Datos Ejemplo:</span>
                <br>  Largo Sépalo:  <input type="number" class="form-control" id="inp1" value="5.0"/>
                <br>  Ancho Sépalo:  <input type="number" class="form-control" id="inp2" value="3.6" />
                <br>  Largo Pétalo:  <input type="number" class="form-control" id="inp3" value="1.4"/>
                <br>  Ancho Pétalo:  <input type="number" class="form-control" id="inp4" value="0.2" />
                <button type="button" onclick="clasificarEjemplo()">Clasificar</button>
                <br>
                <span class="mx-2">-Resultado Modelo:</span> <span id="resultText"></span>
                <br> <br> <br>
                <span class="mr-2">-Probar Modelo:</span>
                <br>
                <textarea id="datosPrueba" cols="40" rows="5"></textarea>
                <button type="button" onclick="probarModelo()">Probar modelo</button>
                <button type="button" onclick="limpiarDatos()">Limpiar datos</button>
                <br>
                <span id="resultPrueba"></span>
                <br> <br> <br>
                 <span class="mx-3">-Mensajes:</span>
                <br>
                <span id="logText"></span>
            </div>
        </div>
        <script>

            let logText = document.getElementById("logText")
            let resText = document.getElementById("resultText")
            let resPrueba = document.getElementById("resultPrueba")

            // funciones auxiliares para mostrar log
            function mostrarFailure(msg) {
                console.error(msg);
                showAlert( "ERROR: "+ msg + "", true);
            }

            function showAlert(msg, mostrarUsuario=false) {
                console.info(msg);
                if (mostrarUsuario) {
                  logText.innerText = logText.innerText + " " + msg + " "
                }
            }

            // función para ejecutar modelo con los datos usando Web API
            function executeModel(datosModel) {
              return new Promise( (resolve, reject) => {
                const xhr = new XMLHttpRequest();
                xhr.open("POST", document.getElementById("URLWebAPI").value, true);
                xhr.setRequestHeader("Content-Type", "application/json");
                xhr.setRequestHeader("ngrok-skip-browser-warning", "69420");
                const body = JSON.stringify({
                        "LargoSepalo": datosModel[0],
                        "AnchoSepalo": datosModel[1],
                        "LargoPetalo": datosModel[2],
                        "AnchoPetalo": datosModel[3]
                      });
                xhr.onload = function () {
                  if (xhr.status == 200) {
                    showAlert("Model API response: " + xhr.responseText);
                    resolve(JSON.parse(xhr.responseText));
                  } else {
                    mostrarFailure("Error al ejecutar Web API: " + xhr.status + " " + xhr.responseText);
                    reject("ERROR");
                  }
                };
                xhr.onerror = function () {
                    mostrarFailure("Error al ejecutar Web API: " + xhr.status + " " + xhr.responseText);
                    reject("ERROR");
                };
                xhr.send(body);
              });
            }

            // función para realizar la clasificación, ejecutando el modelo y formateando el resultado
            async function clasificarIris(datosModel) {
              // ejecuta modelo
              const resModel = await executeModel(datosModel);
              // muestra resultados
              showAlert( "Clasificar: [" + datosModel + "] -> [" + JSON.stringify(resModel) + "]");
              if (resModel == null) {
                  return "ERROR";
              }
              else {
                  return resModel;
              }
            }

            // función para calcular las métricas de eficiencia, precisión y recuperación
            function calcMetricasClasificacion(cantOK, cantMAL, matrizConf) {
              let res = ""
              // calcula eficiencia
              let eff = ( cantOK / (cantOK+cantMAL) ) * 100.0;
              res = res + "-- Cant. OK: " + cantOK + " / Cant. MAL: " + cantMAL + "\\n";
              res = res + "-- Eficiencia General = " + Math.round(eff) + "%\\n";
              // calcula precisión y recuperación por clase
              res = res + "--Métricas por Clase (precisión / recuperación): \\n";
              for (var clase in matrizConf) {
                cantVPa = 0.0;
                cantVPb = 0.0;
                cantFN = 0.0;
                cantFP = 0.0;
                // calcula precisión
                for (var claseR in matrizConf) {
                  if (claseR == clase) {
                    cantVPa += matrizConf[claseR][clase];
                  } else {
                    if (clase in matrizConf[claseR]) {
                      cantFP += matrizConf[claseR][clase];
                    }
                  }
                }
                prec = cantVPa * 100.0 /  (cantVPa + cantFP);
                // calcula recuperación
                for (var claseM in matrizConf[clase]) {
                  if (claseM == clase) {
                    cantVPb += matrizConf[clase][claseM];
                  } else {
                    cantFN += matrizConf[clase][claseM];
                  }
                }
                recl = cantVPb * 100.0 /  (cantVPb + cantFN);
                // muestra
                if (cantVPa != cantVPb) {
                  mostrarFailure("Aborta por error al calcular VP para clase " + clase +":" + cantVPa + " / " + cantVPb + "!");
                  return res;
                }
                res = res + "      clase " + clase + ": P = " + Math.round(prec, 2) + "% / R = " + Math.round(recl, 2) + "%\\n";
              }
              return res;
            }

            // determina clase iris en base a la salida del modelo
            async function clasificarEjemplo() {
              // define datos de entrada
              const x = [ parseFloat(document.getElementById("inp1").value), parseFloat(document.getElementById("inp2").value), parseFloat(document.getElementById("inp3").value), parseFloat(document.getElementById("inp4").value) ];
              const resIris = await clasificarIris(x);
              if (resIris != null) {
                resText.innerText = "--> Clase ID " + resIris.claseID + " - " + resIris.clase
              } else {
                resText.innerText = "NULL!"
              }
            }

            // función para limpiar datos de probar modelo
            function limpiarDatos() {
              document.getElementById("datosPrueba").value = ""
              resPrueba.innerText = ""
            }

            // función para probar modelo
            async function probarModelo() {
              showAlert("Comienza a probar modelo...", true)
              const datos = document.getElementById("datosPrueba").value;
              const lines = datos.split('\\n');
              let linesRes = ""
              let resOK = ""
              let cantOK = 0
              let cantMAL = 0
              let matrizConf = {}
              for(var i = 0; i < lines.length; i++){
                if (lines[i]!="") {
                  let valores = lines[i].split(";");
                  if ( isNaN(valores[0]) || isNaN(valores[1]) || isNaN(valores[2]) || isNaN(valores[3]) ) {
                    showAlert("se descarta de la prueba a " + lines[i], true)
                  }
                  else {
                    let resIris = await clasificarIris( [ parseFloat(valores[0]), parseFloat(valores[1]), parseFloat(valores[2]), parseFloat(valores[3]) ] );
                    if (resIris == null) {
                      mostrarFailure("Aborta por recibir clase NULL!")
                      return
                    }
                    let claseID = resIris.claseID;
                    // registra performance
                    claseModelo = parseFloat(claseID)
                    claseReal = parseFloat(valores[4])
                    if ( claseModelo == claseReal ) {
                        resOK = "OK";
                        cantOK = cantOK + 1;
                    } else {
                        resOK = "MAL";
                        cantMAL = cantMAL + 1;
                    }
                    if (!(claseReal in matrizConf)) {
                      matrizConf[claseReal] = {}
                    }
                    if (!(claseModelo in matrizConf[claseReal])) {
                      matrizConf[claseReal][claseModelo] = 1.0
                    }
                    else {
                      matrizConf[claseReal][claseModelo] += 1.0
                    }
                    linesRes = linesRes + lines[i] + ";Modelo=" + claseID + "->" + resOK + "\\n";
                  }
                }
              }
              document.getElementById("datosPrueba").value = linesRes;
              resPrueba.innerText  =  calcMetricasClasificacion(cantOK, cantMAL, matrizConf);
              showAlert("Fin de probar modelo.", true)
            }

        </script>
</body>

</html>
'''

print("\n* Web Page definida (volver a ejecutar si se necesita realizar algún cambio).\n")


* Web Page definida (volver a ejecutar si se necesita realizar algún cambio).



In [5]:
#@title Habilitar Web API con Flash

import threading
import pandas as pd
from flask import Flask, jsonify, request, current_app
import numpy as np
import logging
import uuid
import os

# crea la application
app = Flask(__name__)

# define parámetros
local_port = "5001" #@param {type:"string"}
nombre_servicio = "/iris" #@param {type:"string"}

# define url local del servicio
urlLocal = "http://127.0.0.1:{}".format(local_port)

print("")
if usar_ngrok_web_publica:
  # Open a ngrok tunnel to the HTTP server

  public_url = ngrok.connect(local_port).public_url
  print(" * ngrok tunnel definido: \"{}\" <-> \"{}\"".format(public_url, urlLocal))

  # Update any base URLs to use the public ngrok URL
  app.config["BASE_URL"] = public_url
  # ... Update inbound traffic via APIs to use the public-facing ngrok URL
  ngrok_public_Web_API = public_url + nombre_servicio
  ngrok_public_Web_PAGE = public_url + "/"
else:
  ngrok_public_Web_API = None
  ngrok_public_Web_PAGE = None
  print("- no se usa ngrok.")

# configura logging en archivo
logFileName = "/content/modelWebAPI.log"
logger = logging.getLogger(__name__)
file_handel = logging.FileHandler(logFileName)
file_handel.setFormatter(logging.Formatter('%(asctime)s %(levelname)s - %(message)s'))
file_handel.setLevel(logging.DEBUG)
logger.addHandler(file_handel)

def mostrarLog():
  stream = os.popen("cat '"+ logFileName +"'")
  output = stream.read()
  print(output)

# define el servicio para clasificar flores IRIS
@app.route(nombre_servicio, methods=["POST"])
def model_web_api():
  try:
    current_app.logger.debug("+ conexion recibida: " + str(uuid.uuid4()))
    ##logger.info('Mostrando los posts del blog')
    # recibe datos de conexion
    data = request.json
    current_app.logger.debug("- datos recibidos: " + str(data))
    # formatea datos como números
    valsInput = np.array( pd.DataFrame(data, index=[0]), dtype=np.float32)
    current_app.logger.debug("- datos convertidos: " + str(valsInput))
    # si está definido el scaler normaliza los datos
    if scaler is not None:
      valsInput = scaler.transform(valsInput)
      current_app.logger.debug("- datos normalizados: " + str(valsInput))
    # ejecuta el modelo con los datos convertidos/normalizados
    resModel = model.predict(valsInput, verbose=0)
    current_app.logger.debug("- resultado del modelo: " + str(resModel))
    # se genera un resultado
    if (len(resModel) == 1):
        # identifica tipo de salida
        if (len(resModel[0]) == 1):
            # como tiene una salida se asume salida lineal (solo la redondea)
            claseID = round(resModel[0])
        else:
            # como tiene validas salidas se asume salida softmax (toma la de mayor puntaje)
            claseID = int( np.argmax(resModel[0], axis=0) )
        current_app.logger.debug("* clase asignada por modelo: " + str(claseID))
        # determina la descripción de la clase
        if (CLASES is not None) and (len(CLASES)>0) and (claseID<len(CLASES)):
          claseDesc = CLASES[claseID]
        else:
          claseDesc = str(claseID)
        return jsonify({"claseID": claseID, "clase": claseDesc})
    else:
        # no se genera un único resultado (error)
        current_app.logger.debug("- resultado del modelo distino de 1 elemento: " + str(resModel))
        return jsonify({"clase": "ERROR"})
  except Exception as error:
     current_app.logger.debug("!! error detectado: " + str(error))

# muestra nombre de APIs
nombre_WebAPI = "{}{}".format(urlLocal, nombre_servicio)
print("\n > Web API local establecida en: ",  nombre_WebAPI)
if usar_ngrok_web_publica and (ngrok_public_Web_API is not None):
  print("\n > ngrok public Web API establecida en: ",  ngrok_public_Web_API)

# define la página web index asociada al servicio
if (DEFAULT_WEB_PAGE is not None) and (DEFAULT_WEB_PAGE!=""):
  # crea página por defecto
  @app.route("/", methods=["GET"])
  def model_web_page():
    return DEFAULT_WEB_PAGE.replace("id=\"URLWebAPI\" value=\"\"", "id=\"URLWebAPI\" value=\"" + ngrok_public_Web_API + "\"")
  # muestra nombre de la WEB PAGE
  nombre_WebPage = "{}{}".format(urlLocal, "/")
  print("\n > Web Page local establecida en: ",  nombre_WebPage)
  if (ngrok_public_Web_PAGE is not None):
    print("\n > ngrok public Web Page establecida en: ",  ngrok_public_Web_PAGE)
print("")

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False, "port" : local_port, "debug": True}).start()

# espera 5 segundos a que arranque el servicio
import time
time.sleep(5)


 * ngrok tunnel definido: "https://aff3-34-74-225-24.ngrok-free.app" <-> "http://127.0.0.1:5001"

 > Web API local establecida en:  http://127.0.0.1:5001/iris

 > ngrok public Web API establecida en:  https://aff3-34-74-225-24.ngrok-free.app/iris

 > Web Page local establecida en:  http://127.0.0.1:5001/

 > ngrok public Web Page establecida en:  https://aff3-34-74-225-24.ngrok-free.app/

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit


# Probar Web API local

In [11]:
#@title Probar Web API local con un ejemplo

import os
import json

LargoSepalo = 5.5 #@param{type:"number"}
AnchoSepalo = 2.6 #@param{type:"number"}
LargoPetalo = 4.4 #@param{type:"number"}
AnchoPetalo = 1.2 #@param{type:"number"}

dictValues = {
    "LargoSepalo" : LargoSepalo,
    "AnchoSepalo" : AnchoSepalo,
    "LargoPetalo" : LargoPetalo,
    "AnchoPetalo" : AnchoPetalo
    }

# función auxiliar para probar API
def ejecutarModelWebAPI(dictValues, URL_API, mostrarRes=True):
  # ejecuta la web API usando curl
  cmdStr = "curl -d '" + str(dictValues).replace("'", '"') + "' -H \"Content-Type: application/json\" -X POST " + URL_API
  if mostrarRes:
    print("\n", cmdStr, "\n")
  # ejecuta y muestra resultados
  stream = os.popen(cmdStr)
  output = stream.read()
  if mostrarRes:
    print("\n --> ", output)
  # devuelve la clase
  return json.loads(output)

# ejecuta API
print("")
print("> Datos: ", dictValues)
print("> Resultado: ", ejecutarModelWebAPI(dictValues, nombre_WebAPI) )
print("")


DEBUG:__main__:+ conexion recibida: 24dd8682-64fb-49cd-b4f2-20f8ae2199f2
DEBUG:__main__:- datos recibidos: {'LargoSepalo': 5.5, 'AnchoSepalo': 2.6, 'LargoPetalo': 4.4, 'AnchoPetalo': 1.2}
DEBUG:__main__:- datos convertidos: [[5.5 2.6 4.4 1.2]]
DEBUG:__main__:- resultado del modelo: [[3.0309169e-05 3.4290313e-04 8.9357340e-01 1.0605336e-01]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 15:11:50] "POST /iris HTTP/1.1" 200 -



> Datos:  {'LargoSepalo': 5.5, 'AnchoSepalo': 2.6, 'LargoPetalo': 4.4, 'AnchoPetalo': 1.2}

 curl -d '{"LargoSepalo": 5.5, "AnchoSepalo": 2.6, "LargoPetalo": 4.4, "AnchoPetalo": 1.2}' -H "Content-Type: application/json" -X POST http://127.0.0.1:5001/iris 


 -->  {
  "clase": "Versicolor",
  "claseID": 2
}

> Resultado:  {'clase': 'Versicolor', 'claseID': 2}



In [12]:
#@title Probar Web API local con ejemplos de un CSV que tiene los datos mostrados en <construir-RNA-MLP-IRIS.ipynb>
from google.colab import files

separador_valores = ";" #@param{type:"string"}
if separador_valores == "":
  separador_valores = ";"

def cargarArchivo(fn):

  # lo carga en lista
  uploadedData = []
  with open(fn, 'r', encoding='utf-8') as f:
    contents = f.readlines()
  # separa en lineas
  uploadedData.extend( ("\n".join(contents)).split("\n") )

  print('-- Archivo "{name}" con largo {length} bytes cargado'.format(
    name=fn, length=len(uploaded[fn])))

  return uploadedData

# sube archivo
uploaded = files.upload()
# procesa los datos
uploadedData = []
for fn in uploaded.keys():
    uploadedData.extend( cargarArchivo(fn))

if len(uploadedData)>0:
  print("\n")
  # procesa los datos cargados
  classReal = []
  classPreds = []
  for data in uploadedData:
    if data!="":
      arAux = data.split(separador_valores)
      dictValues = {
      "LargoSepalo" : arAux[0],
      "AnchoSepalo" : arAux[1],
      "LargoPetalo" : arAux[2],
      "AnchoPetalo" : arAux[3]
      }
      classReal.append( arAux[4] )
      claseModelo = ejecutarModelWebAPI(dictValues, nombre_WebAPI, mostrarRes=False)
      if "claseID" in claseModelo:
        classPreds.append( str(claseModelo["claseID"]) )
      else:
        print("Clase no identificada: ", dictValues, "->", claseModelo)
        classPreds.append( "ERROR" )
  print("\n")

  from sklearn.metrics import classification_report
  from sklearn.metrics import confusion_matrix
  import pandas as pd

  # muestra reporte de clasificación
  print("\n Reporte de Clasificación: ")
  print(classification_report(classReal, classPreds))

  # muestra matriz de confusion
  print('\nMatriz de Confusión ( real / modelo ): ')
  cm = confusion_matrix(classReal, classPreds)
  cmtx = pd.DataFrame(
      cm
    )
  # agrega para poder mostrar la matrix de confusión completa
  pd.options.display.max_rows = 100
  pd.options.display.max_columns = 100
  cmtx.sort_index(axis=0, inplace=True)
  cmtx.sort_index(axis=1, inplace=True)
  print(cmtx)
  print("\n")


DEBUG:__main__:+ conexion recibida: 868b7392-eabd-4f1f-936d-7a16e414ebc0
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '7.2', 'AnchoSepalo': '3.0', 'LargoPetalo': '5.8', 'AnchoPetalo': '1.6'}
DEBUG:__main__:- datos convertidos: [[7.2 3.  5.8 1.6]]
DEBUG:__main__:- resultado del modelo: [[9.0679220e-07 2.5120041e-06 3.6179799e-01 6.3819861e-01]]
DEBUG:__main__:* clase asignada por modelo: 3
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 15:12:07] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: 7aaed37e-5679-4239-b5f5-917ae906292f
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '6.6', 'AnchoSepalo': '2.9', 'LargoPetalo': '4.6', 'AnchoPetalo': '1.3'}
DEBUG:__main__:- datos convertidos: [[6.6 2.9 4.6 1.3]]


Saving datos_PRUEBA.csv to datos_PRUEBA.csv
-- Archivo "datos_PRUEBA.csv" con largo 684 bytes cargado




DEBUG:__main__:- resultado del modelo: [[3.1368022e-06 5.6778977e-04 9.9396753e-01 5.4615685e-03]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 15:12:07] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: c0e5b2e8-768f-4b0d-8980-5318cf5f6776
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '5.1', 'AnchoSepalo': '2.5', 'LargoPetalo': '3.0', 'AnchoPetalo': '1.1'}
DEBUG:__main__:- datos convertidos: [[5.1 2.5 3.  1.1]]
DEBUG:__main__:- resultado del modelo: [[4.7793314e-05 4.4225968e-02 9.5536202e-01 3.6417038e-04]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [05/Aug/2024 15:12:07] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: 324494b1-8640-4090-ad48-75398cf8487a
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '4.6', 'AnchoSepalo': '3.4', 'LargoPetalo': '1.4', 'AnchoPetalo': '0.3'}
DEBUG:__main__:- datos convertidos: [[4.6 3.4 1.4 0.3]]
DEBUG:__main__:- resultado del modelo: [[1.33899




 Reporte de Clasificación: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        13
           2       1.00      0.92      0.96        13
           3       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38


Matriz de Confusión ( real / modelo ): 
    0   1   2
0  13   0   0
1   0  12   1
2   0   0  12




In [13]:
#@title Mostrar Archivo de Log (nuevamente)
print("")
mostrarLog()
print("")


2024-08-05 15:07:49,160 DEBUG - + conexion recibida: 90b597e7-c2c4-45f0-9249-775e64b9c028
2024-08-05 15:07:49,164 DEBUG - - datos recibidos: {'LargoSepalo': 5.5, 'AnchoSepalo': 2.6, 'LargoPetalo': 4.4, 'AnchoPetalo': 1.2}
2024-08-05 15:07:49,169 DEBUG - - datos convertidos: [[5.5 2.6 4.4 1.2]]
2024-08-05 15:07:49,170 DEBUG - !! error detectado: name 'scaler' is not defined
2024-08-05 15:08:09,082 DEBUG - + conexion recibida: 3a2d323c-a793-4f43-89dc-781afcd89f07
2024-08-05 15:08:09,084 DEBUG - - datos recibidos: {'LargoSepalo': 5, 'AnchoSepalo': 3.6, 'LargoPetalo': 1.4, 'AnchoPetalo': 0.2}
2024-08-05 15:08:09,094 DEBUG - - datos convertidos: [[5.  3.6 1.4 0.2]]
2024-08-05 15:08:09,095 DEBUG - !! error detectado: name 'scaler' is not defined
2024-08-05 15:11:50,631 DEBUG - + conexion recibida: 24dd8682-64fb-49cd-b4f2-20f8ae2199f2
2024-08-05 15:11:50,633 DEBUG - - datos recibidos: {'LargoSepalo': 5.5, 'AnchoSepalo': 2.6, 'LargoPetalo': 4.4, 'AnchoPetalo': 1.2}
2024-08-05 15:11:50,639 DEB